In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import layers, utils, init, learn, explore
from tfomics import neuralnetwork as nn
from tfomics.build_network import *

# import models
from model_zoo import fourthplace_connectomics_model
from model_zoo import simple_connectomics_model
from model_zoo import simple_connectomics_model2
from model_zoo import residual_connectomics_model
from model_zoo import residual_connectomics_model2

# load data

In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf
y_train = ltf
X_valid = dtf_crossval
y_valid = ltf_crossval

CPU times: user 504 ms, sys: 1.18 s, total: 1.68 s
Wall time: 1.68 s


# Simple connectomics model

In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, [placeholders['inputs']])
nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 18)
layer3: conv1_batch
(?, 2, 326, 18)
layer4: conv1_active
(?, 2, 326, 18)
layer5: conv2
(?, 1, 322, 40)
layer6: conv2_batch
(?, 1, 322, 40)
layer7: conv2_active
(?, 1, 322, 40)
layer8: conv2_pool
(?, 1, 32, 40)
layer9: conv3
(?, 1, 32, 15)
layer10: conv3_batch
(?, 1, 32, 15)
layer11: conv3_active
(?, 1, 32, 15)
layer12: dense1
(?, 100)
layer13: dense1_bias
(?, 100)
layer14: dense1_active
(?, 100)
layer15: dense1_dropout
(?, 100)
layer16: dense2
(?, 2)
layer17: dense2_bias
(?, 2)
layer18: dense2_active
(?, 2)
layer19: output
(?, 2)
----------------------------------------------------------------------------


In [4]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}

sess = tf.Session()
sess.run(tf.initialize_all_variables())
learn.train_minibatch(sess, nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=500, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=1000, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()


sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=1500, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()


sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=2000, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()



Epoch 1 out of 20 
[==============================] 100.0% -- time=0s -- loss=0.38461 -- accuracy=85.07%  
  valid loss:		0.35943
  valid accuracy:	0.85784+/-0.00000
  valid auc-roc:	0.91697+/-0.00000
  valid auc-pr:		0.91051+/-0.00554
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_best.ckpt
Epoch 2 out of 20 
[==============================] 100.0% -- time=0s -- loss=0.36105 -- accuracy=85.87%  
  valid loss:		0.35419
  valid accuracy:	0.85993+/-0.00003
  valid auc-roc:	0.91999+/-0.00000
  valid auc-pr:		0.91390+/-0.00535
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_best.ckpt
Epoch 3 out of 20 
[==============================] 100.0% -- time=0s -- loss=0.35562 -- accuracy=86.10%  
  valid loss:		0.35098
  valid accuracy:	0.86132+/-0.00000
  valid auc-roc:	0.92202+/-0.00000
  valid auc-pr:		0.91656+/-0.00432
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_best.ckpt
Epoch 4 out of 20 
[===========

# Simple connectomics model

In [5]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, [placeholders['inputs']])
nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version2'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 32)
layer3: conv1_batch
(?, 2, 326, 32)
layer4: conv1_active
(?, 2, 326, 32)
layer5: conv2
(?, 1, 322, 64)
layer6: conv2_batch
(?, 1, 322, 64)
layer7: conv2_active
(?, 1, 322, 64)
layer8: conv2_pool
(?, 1, 32, 64)
layer9: conv3
(?, 1, 32, 128)
layer10: conv3_batch
(?, 1, 32, 128)
layer11: conv3_active
(?, 1, 32, 128)
layer12: dense1
(?, 256)
layer13: dense1_bias
(?, 256)
layer14: dense1_active
(?, 256)
layer15: dense1_dropout
(?, 256)
layer16: dense2
(?, 2)
layer17: dense2_bias
(?, 2)
layer18: dense2_active
(?, 2)
layer19: output
(?, 2)
----------------------------------------------------------------------------


In [6]:
# run session
sess = tf.Session()

# initialize variables
sess.run(tf.initialize_all_variables())

In [7]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}

sess = tf.Session()
sess.run(tf.initialize_all_variables())
learn.train_minibatch(sess, nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=500, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=1000, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()



Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.41484 -- accuracy=85.12%  
  valid loss:		0.36046
  valid accuracy:	0.85751+/-0.00001
  valid auc-roc:	0.91875+/-0.00000
  valid auc-pr:		0.91293+/-0.00520
lower cross-validation found
saving model to:  ../results/tf-test/1d_version2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.36393 -- accuracy=85.92%  
  valid loss:		0.35547
  valid accuracy:	0.86139+/-0.00001
  valid auc-roc:	0.92200+/-0.00000
  valid auc-pr:		0.91672+/-0.00460
lower cross-validation found
saving model to:  ../results/tf-test/1d_version2_best.ckpt
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.35531 -- accuracy=86.16%  
  valid loss:		0.34755
  valid accuracy:	0.86272+/-0.00001
  valid auc-roc:	0.92442+/-0.00001
  valid auc-pr:		0.91981+/-0.00397
lower cross-validation found
saving model to:  ../results/tf-test/1d_version2_best.ckpt
Epoch 4 out of 50 
[========

# residual model

In [8]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, [placeholders['inputs']])

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version_residual'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath=filepath)

In [9]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}

sess = tf.Session()
sess.run(tf.initialize_all_variables())
learn.train_minibatch(sess, nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=500, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=1000, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()



Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.39658 -- accuracy=84.92%  
  valid loss:		0.36943
  valid accuracy:	0.85824+/-0.00001
  valid auc-roc:	0.91620+/-0.00000
  valid auc-pr:		0.90966+/-0.00675
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_residual_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.36112 -- accuracy=85.94%  
  valid loss:		0.35876
  valid accuracy:	0.85977+/-0.00075
  valid auc-roc:	0.92230+/-0.00001
  valid auc-pr:		0.91732+/-0.00435
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_residual_best.ckpt
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.35143 -- accuracy=86.26%  
  valid loss:		0.35093
  valid accuracy:	0.86147+/-0.00001
  valid auc-roc:	0.92502+/-0.00000
  valid auc-pr:		0.92074+/-0.00376
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_residual_best.ckpt
Epoc

# residual model 2

In [10]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, [placeholders['inputs']])

# set output file paths
resultspath = utils.make_directory('../results', 'tf-test')
output_name = '1d_version_residual2'
filepath = os.path.join(resultspath, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath=filepath)

In [11]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}

sess = tf.Session()
sess.run(tf.initialize_all_variables())
learn.train_minibatch(sess, nntrainer, data, batch_size=100, num_epochs=50, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=500, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()

sess = tf.Session()
sess.run(tf.initialize_all_variables())
nntrainer.set_best_parameters(sess)
learn.train_minibatch(sess, nntrainer, data, batch_size=1000, num_epochs=20, 
                    patience=5, verbose=1, shuffle=True)
sess.close()


Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.39804 -- accuracy=84.92%  
  valid loss:		0.36821
  valid accuracy:	0.85858+/-0.00001
  valid auc-roc:	0.91610+/-0.00001
  valid auc-pr:		0.90923+/-0.00626
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_residual2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.36300 -- accuracy=85.99%  
  valid loss:		0.36488
  valid accuracy:	0.86188+/-0.00001
  valid auc-roc:	0.92204+/-0.00000
  valid auc-pr:		0.91672+/-0.00380
lower cross-validation found
saving model to:  ../results/tf-test/1d_version_residual2_best.ckpt
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.35261 -- accuracy=86.30%  
  valid loss:		0.37900
  valid accuracy:	0.85603+/-0.00001
  valid auc-roc:	0.92518+/-0.00000
  valid auc-pr:		0.92089+/-0.00351
Epoch 4 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.34502 -- accuracy=